<a href="https://colab.research.google.com/github/9soohyun/24-1-SMBA/blob/main/3%EC%9E%A5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

base_src = 'drive/MyDrive/Colab Notebooks/추천시스템'
u_user_src = os.path.join(base_src, 'u.user')
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users = pd.read_csv(u_user_src,
                    sep='|',
                    names=u_cols,
                    encoding = 'latin-1')
users = users.set_index('user_id')

u_item_src = os.path.join(base_src, 'u.item')
i_cols = ['movie_id','title','release date','video release date',
          'IMDB URL','unknown','Action','Adventure','Animation',
          'Children\'s','Comedy','Crime','Documentary','Drama','Fantasy',
          'Film-Noir','Horror','Musical','Mystery','Romance','Sci-Fi','Thriller','War','Western']
movies = pd.read_csv(u_item_src,
                    sep='|',
                    names=i_cols,
                    encoding = 'latin-1')
movies = movies.set_index('movie_id')

u_data_src = os.path.join(base_src, 'u.data')
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(u_data_src,
                    sep='\t',
                    names=r_cols,
                    encoding = 'latin-1')
# ratings = ratings.set_index('user_id')

In [2]:
def RMSE (y_true, y_pred):
  return np.sqrt(np.mean((np.array(y_true)-np.array(y_pred))**2))

def score(model):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user,movie) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

x = ratings.copy()
y = ratings['user_id']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)
ratings_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
matrix_dummy = ratings_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index = ratings_matrix.index, columns=ratings_matrix.index)

In [4]:
def CF_simple(user_id, movie_id):
  if movie_id in ratings_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)
    mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
  else:
    mean_rating = 3.0
  return mean_rating

score(CF_simple)

1.0225980179865148

In [5]:
def score(model, neighbor_size=0):
  id_pairs = zip(x_test['user_id'], x_test['movie_id'])
  y_pred = np.array([model(user,movie, neighbor_size) for (user, movie) in id_pairs])
  y_true = np.array(x_test['rating'])
  return RMSE(y_true, y_pred)

In [6]:
def CF_knn(user_id, movie_id, neighbor_size = 0):
  if movie_id in ratings_matrix.columns:
    sim_scores = user_similarity[user_id].copy()
    movie_ratings = ratings_matrix[movie_id].copy()
    none_rating_idx = movie_ratings[movie_ratings.isnull()].index
    movie_ratings = movie_ratings.dropna()
    sim_scores = sim_scores.drop(none_rating_idx)

    if neighbor_size == 0:
      mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()

    else:
      if len(sim_scores) >1:
        neighbor_size = min(neighbor_size, len(sim_scores))
        sim_scores = np.array(sim_scores)
        movie_ratings = np.array(movie_ratings)
        user_idx = np.argsort(sim_scores)
        sim_scores = sim_scores[user_idx][-neighbor_size:]
        movie_ratings = movie_ratings[user_idx][-neighbor_size:]
        mean_rating = np.dot(sim_scores, movie_ratings) / sim_scores.sum()
      else:
        mean_rating = 3.0
  else:
    mean_rating = 3.0

  return mean_rating

score(CF_knn, neighbor_size=30)

1.0133484062322007

In [7]:
rating_matrix = ratings.pivot_table(values='rating', index = 'user_id', columns = 'movie_id')
matrix_dummy = rating_matrix.copy().fillna(0)
user_similarity = cosine_similarity(matrix_dummy, matrix_dummy)
user_similarity = pd.DataFrame(user_similarity, index=rating_matrix.index, columns=rating_matrix.index)

def recom_movie(user_id, n_items, neighbor_size = 30):
  user_movie = rating_matrix.loc[user_id].copy()

  for movie in rating_matrix.columns:
    if pd.notnull(user_movie.loc[movie]):
      user_movie.loc[movie] = 0
    else:
      user_movie.loc[movie] = CF_knn(user_id, movie, neighbor_size)

  movie_sort = user_movie.sort_values(ascending=False)[:n_items]
  recom_movies = movies.loc[movie_sort.index]
  recommendations = recom_movies['title']
  return recommendations

In [8]:
recom_movie(user_id=729, n_items=5, neighbor_size=30)

movie_id
1189                      Prefontaine (1997)
1467    Saint of Fort Washington, The (1993)
1064                        Crossfire (1947)
1293                         Star Kid (1997)
1516                Wedding Gift, The (1994)
Name: title, dtype: object

In [9]:
for neighbor_size in[10, 20, 30, 40, 50, 60]:
  print('Neighbor size = %d : RMSE = %.4f' %(neighbor_size, score(CF_knn, neighbor_size)))

Neighbor size = 10 : RMSE = 1.0193
Neighbor size = 20 : RMSE = 1.0076
Neighbor size = 30 : RMSE = 1.0069
Neighbor size = 40 : RMSE = 1.0069
Neighbor size = 50 : RMSE = 1.0080
Neighbor size = 60 : RMSE = 1.0094
